# Retail Sales Analytics 2024 — End-to-End Project

This notebook generates a synthetic retail dataset, performs analysis, creates charts, exports a PDF report and PowerPoint deck, and prepares SQL scripts. Run the cells in order to reproduce all artifacts.

**Outputs produced:**
- `retail_sales_2024.csv`
- `retail_sales_schema.sql`
- `retail_sales_inserts.sql`
- `retail_sales_analysis_queries.sql`
- `retail_sales_report_2024.pdf`
- `retail_sales_presentation_2024.pptx`
- `README.md`
- `retail_sales_project_bundle.zip`


## 1. Setup & Imports

In [ ]:

import os, random, math
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Optional: PowerPoint
pptx_available = True
try:
    from pptx import Presentation
    from pptx.util import Inches
except Exception as e:
    pptx_available = False

np.random.seed(42)
random.seed(42)

out_dir = "."  # write outputs to current folder
fig_dir = os.path.join(out_dir, "figs")
os.makedirs(fig_dir, exist_ok=True)


## 2. Generate Synthetic Dataset

In [ ]:

start_date = datetime(2024,1,1)
end_date   = datetime(2024,12,31)
days = (end_date - start_date).days + 1
dates = [start_date + timedelta(days=i) for i in range(days)]

regions  = ["North", "South", "East", "West"]
segments = ["Consumer", "Corporate", "Home Office", "Small Business"]
categories = {
    "Technology": ["Laptop", "Mouse", "Keyboard", "Monitor", "Headphones"],
    "Furniture": ["Chair", "Desk", "Bookcase", "Table", "Cabinet"],
    "Office Supplies": ["Paper", "Binder", "Pen", "Stapler", "Calendar"],
}
base_price = {
    "Laptop": 900, "Mouse": 20, "Keyboard": 30, "Monitor": 200, "Headphones": 80,
    "Chair": 120, "Desk": 300, "Bookcase": 150, "Table": 250, "Cabinet": 200,
    "Paper": 5, "Binder": 6, "Pen": 2, "Stapler": 8, "Calendar": 10
}

rows = []
order_id = 10000
for dt in dates:
    weekday = dt.weekday()
    base_orders = 30 if weekday < 5 else 12
    month = dt.month
    season_boost = 1.25 if month in [8,9] else (1.5 if month in [11,12] else 1.0)
    n_orders = int(np.random.poisson(base_orders * season_boost))
    for _ in range(n_orders):
        region = random.choice(regions)
        segment = random.choice(segments)
        category = random.choice(list(categories.keys()))
        product = random.choice(categories[category])
        units = max(1, int(np.random.lognormal(mean=0.4, sigma=0.6)))
        unit_price = base_price[product] * np.random.uniform(0.85, 1.15)
        revenue = units * unit_price
        cost = revenue * np.random.uniform(0.6, 0.8)
        discount = float(np.clip(np.random.normal(loc=0.05 if month in [11,12] else 0.02, scale=0.02), 0, 0.3))
        revenue_after_discount = revenue * (1 - discount)
        profit = revenue_after_discount - cost
        rows.append({
            "order_id": order_id,
            "order_date": dt.date().isoformat(),
            "region": region,
            "segment": segment,
            "category": category,
            "product": product,
            "units": int(units),
            "unit_price": round(unit_price, 2),
            "discount": round(discount, 3),
            "revenue": round(revenue_after_discount, 2),
            "cost": round(cost, 2),
            "profit": round(profit, 2)
        })
        order_id += 1

df = pd.DataFrame(rows)
csv_path = os.path.join(out_dir, "retail_sales_2024.csv")
df.to_csv(csv_path, index=False)
df.head()


## 3. KPIs & Aggregations

In [ ]:

df['order_date'] = pd.to_datetime(df['order_date'])
df['month'] = df['order_date'].dt.to_period('M').dt.to_timestamp()

kpi = {
    "Total Revenue": df["revenue"].sum(),
    "Total Profit": df["profit"].sum(),
    "Avg Order Value": df["revenue"].mean(),
    "Orders": len(df),
    "Units Sold": df["units"].sum(),
    "Profit Margin %": (df["profit"].sum()/df["revenue"].sum())*100.0
}

rev_by_month = df.groupby('month')['revenue'].sum().reset_index()
profit_by_region = df.groupby('region')['profit'].sum().reset_index()
top_products = df.groupby('product')['revenue'].sum().sort_values(ascending=False).head(10).reset_index()
cat_mix = df.groupby('category')['revenue'].sum().reset_index()
segment_rev = df.groupby('segment')['revenue'].sum().reset_index()

kpi


## 4. Charts (matplotlib only)

In [ ]:

import matplotlib.pyplot as plt

def save_line_chart(x, y, title, xlab, ylab, path):
    plt.figure()
    plt.plot(x, y, marker='o')
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

def save_bar_chart(labels, values, title, xlab, ylab, path, rotation=0):
    plt.figure()
    plt.bar(labels, values)
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks(rotation=rotation, ha='right' if rotation else 'center')
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

fig_paths = {}
fig_paths["rev_month"] = os.path.join(fig_dir, "revenue_by_month.png")
save_line_chart(rev_by_month['month'].dt.strftime('%Y-%m'), rev_by_month['revenue'],
                "Revenue by Month (2024)", "Month", "Revenue", fig_paths["rev_month"])

fig_paths["profit_region"] = os.path.join(fig_dir, "profit_by_region.png")
save_bar_chart(profit_by_region['region'], profit_by_region['profit'],
               "Profit by Region", "Region", "Profit", fig_paths["profit_region"])

fig_paths["top_products"] = os.path.join(fig_dir, "top10_products_by_revenue.png")
save_bar_chart(top_products['product'], top_products['revenue'],
               "Top 10 Products by Revenue", "Product", "Revenue", fig_paths["top_products"], rotation=30)

fig_paths["cat_mix"] = os.path.join(fig_dir, "category_mix.png")
save_bar_chart(cat_mix['category'], cat_mix['revenue'],
               "Revenue by Category", "Category", "Revenue", fig_paths["cat_mix"])

fig_paths["segment_rev"] = os.path.join(fig_dir, "segment_revenue.png")
save_bar_chart(segment_rev['segment'], segment_rev['revenue'],
               "Revenue by Segment", "Segment", "Revenue", fig_paths["segment_rev"])

list(fig_paths.items())


## 5. PDF Report

In [ ]:

pdf_path = os.path.join(out_dir, "retail_sales_report_2024.pdf")
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(pdf_path) as pdf:
    plt.figure(figsize=(8.27, 11.69)); plt.axis('off')
    title = "Retail Sales Analysis 2024\nEnd-to-End Data Analytics Project"
    subtitle = f"Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    lines = ["Contents:", "• Dataset (CSV)", "• SQL schema & analysis queries", "• Visual insights (charts)", "• Executive summary"]
    plt.text(0.5, 0.8, title, ha='center', va='center', fontsize=20, weight='bold')
    plt.text(0.5, 0.76, subtitle, ha='center', va='center', fontsize=10)
    for i, ln in enumerate(lines):
        plt.text(0.1, 0.65 - i*0.04, ln, fontsize=12)
    pdf.savefig(); plt.close()

    # KPI page
    plt.figure(figsize=(8.27,11.69)); plt.axis('off')
    plt.text(0.1, 0.95, "Key Metrics", fontsize=18, weight='bold')
    y = 0.88
    for k, v in kpi.items():
        disp = f"{v:,.2f}%" if "Percent" in k else (f"{v:,.2f}" if isinstance(v, float) else f"{v:,}")
        plt.text(0.1, y, f"{k}: {disp}", fontsize=12); y -= 0.04
    pdf.savefig(); plt.close()

    for key, path in fig_paths.items():
        img = plt.imread(path)
        plt.figure(figsize=(11.69, 8.27)); plt.imshow(img); plt.axis('off')
        pdf.savefig(); plt.close()

    plt.figure(figsize=(8.27, 11.69)); plt.axis('off')
    insights = [
        f"Total revenue: {kpi['Total Revenue']:,.2f} with profit margin {kpi['Profit Margin %']:.2f}%.",
        "Seasonality peaks around Aug–Sep and Nov–Dec.",
        "Technology leads revenue; Laptop & Monitor drive sales.",
        "West & North regions lead profit; grow South/East.",
        "Consumer & Corporate segments contribute most revenue."
    ]
    plt.text(0.1, 0.95, "Executive Insights", fontsize=18, weight='bold')
    y=0.88
    for ln in insights:
        plt.text(0.1, y, f"• {ln}", fontsize=12); y -= 0.05
    pdf.savefig(); plt.close()

pdf_path


## 6. SQL Schema, Inserts, and Analysis Queries

In [ ]:

schema_sql_path = os.path.join(out_dir, "retail_sales_schema.sql")
inserts_sql_path = os.path.join(out_dir, "retail_sales_inserts.sql")
analysis_sql_path = os.path.join(out_dir, "retail_sales_analysis_queries.sql")

schema_sql = """
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
    order_id INTEGER PRIMARY KEY,
    order_date DATE,
    region TEXT,
    segment TEXT,
    category TEXT,
    product TEXT,
    units INTEGER,
    unit_price REAL,
    discount REAL,
    revenue REAL,
    cost REAL,
    profit REAL
);
"""
with open(schema_sql_path, "w", encoding="utf-8") as f:
    f.write(schema_sql.strip() + "\n")

with open(inserts_sql_path, "w", encoding="utf-8") as f:
    f.write("-- Inserts for sales table\nBEGIN TRANSACTION;\n")
    for _, r in df.iterrows():
        vals = (
            int(r['order_id']),
            (r['order_date'].date().isoformat() if isinstance(r['order_date'], pd.Timestamp) else str(r['order_date'])),
            str(r['region']).replace("'", "''"),
            str(r['segment']).replace("'", "''"),
            str(r['category']).replace("'", "''"),
            str(r['product']).replace("'", "''"),
            int(r['units']),
            float(r['unit_price']),
            float(r['discount']),
            float(r['revenue']),
            float(r['cost']),
            float(r['profit'])
        )
        f.write(f"INSERT INTO sales (order_id, order_date, region, segment, category, product, units, unit_price, discount, revenue, cost, profit) VALUES ({vals[0]}, '{vals[1]}', '{vals[2]}', '{vals[3]}', '{vals[4]}', '{vals[5]}', {vals[6]}, {vals[7]:.2f}, {vals[8]:.3f}, {vals[9]:.2f}, {vals[10]:.2f}, {vals[11]:.2f});\n")
    f.write("COMMIT;\n")

analysis_sql = """
-- Monthly revenue trend
SELECT strftime('%Y-%m', order_date) AS month, SUM(revenue) AS total_revenue
FROM sales
GROUP BY 1
ORDER BY 1;

-- Profit by region
SELECT region, SUM(profit) AS total_profit
FROM sales
GROUP BY region
ORDER BY total_profit DESC;

-- Top 10 products by revenue
SELECT product, SUM(revenue) AS revenue
FROM sales
GROUP BY product
ORDER BY revenue DESC
LIMIT 10;

-- Category mix
SELECT category, SUM(revenue) AS revenue
FROM sales
GROUP BY category
ORDER BY revenue DESC;

-- Segment revenue and margin
SELECT segment, SUM(revenue) AS revenue, SUM(profit) AS profit,
       ROUND(100.0 * SUM(profit)/NULLIF(SUM(revenue),0), 2) AS margin_pct
FROM sales
GROUP BY segment
ORDER BY revenue DESC;

-- Average order value
SELECT AVG(revenue) AS avg_order_value
FROM sales;
"""
with open(analysis_sql_path, "w", encoding="utf-8") as f:
    f.write(analysis_sql.strip() + "\n")

schema_sql_path, inserts_sql_path, analysis_sql_path


## 7. PowerPoint Deck

In [ ]:

ppt_path = os.path.join(out_dir, "retail_sales_presentation_2024.pptx")
if pptx_available:
    prs = Presentation()
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    slide.shapes.title.text = "Retail Sales Analysis 2024"
    slide.placeholders[1].text = "End-to-End Data Analytics Project"
    # KPIs
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    slide.shapes.title.text = "Key Metrics"
    kpi_lines = [
        f"Total Revenue: {kpi['Total Revenue']:,.2f}",
        f"Total Profit: {kpi['Total Profit']:,.2f}",
        f"Avg Order Value: {kpi['Avg Order Value']:,.2f}",
        f"Orders: {kpi['Orders']:,}",
        f"Units Sold: {kpi['Units Sold']:,}",
        f"Profit Margin: {kpi['Profit Margin %']:.2f}%"
    ]
    slide.placeholders[1].text = "\n".join(kpi_lines)

    def add_image_slide(prs, title, img_path):
        slide = prs.slides.add_slide(prs.slide_layouts[5])
        slide.shapes.title.text = title
        from pptx.util import Inches
        left, top, height = Inches(1), Inches(1.5), Inches(5)
        slide.shapes.add_picture(img_path, left, top, height=height)

    add_image_slide(prs, "Revenue by Month", fig_paths["rev_month"])
    add_image_slide(prs, "Profit by Region", fig_paths["profit_region"])
    add_image_slide(prs, "Top 10 Products by Revenue", fig_paths["top_products"])
    add_image_slide(prs, "Revenue by Category", fig_paths["cat_mix"])
    add_image_slide(prs, "Revenue by Segment", fig_paths["segment_rev"])

    # Insights
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    slide.shapes.title.text = "Executive Insights"
    slide.placeholders[1].text = (
        "• Seasonality peaks in Aug–Sep and Nov–Dec.\n"
        "• Technology leads revenue; Laptop & Monitor are top drivers.\n"
        "• West & North regions lead profit; focus growth in South/East.\n"
        "• Consumer & Corporate are key segments."
    )
    prs.save(ppt_path)
else:
    with open(ppt_path.replace(".pptx", "_NOTE.txt"), "w") as f:
        f.write("python-pptx not available; PPTX not generated.")
ppt_path


## 8. README and ZIP bundle

In [ ]:

readme_text = f"""# Retail Sales Analytics 2024

This is a complete end-to-end data analytics mini-project with synthetic retail sales data.
Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.

## Files Included
- retail_sales_2024.csv
- retail_sales_schema.sql
- retail_sales_inserts.sql
- retail_sales_analysis_queries.sql
- retail_sales_report_2024.pdf
- retail_sales_presentation_2024.pptx
- README.md
- retail_sales_project_bundle.zip (this bundle)

## Usage
- Load the CSV into your analytics tool of choice (Python, R, Excel).
- Execute the SQL files in SQLite/Postgres/MySQL to recreate the dataset and run analysis queries.
- Share the PDF and present the PPTX to stakeholders.
- Re-run this notebook to regenerate everything end-to-end.
"""
readme_path = os.path.join(out_dir, "README.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_text)

# Build the ZIP bundle
import zipfile
zip_path = os.path.join(out_dir, "retail_sales_project_bundle.zip")
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for path in [
        os.path.join(out_dir,"retail_sales_2024.csv"),
        os.path.join(out_dir,"retail_sales_schema.sql"),
        os.path.join(out_dir,"retail_sales_inserts.sql"),
        os.path.join(out_dir,"retail_sales_analysis_queries.sql"),
        os.path.join(out_dir,"retail_sales_report_2024.pdf"),
        os.path.join(out_dir,"retail_sales_presentation_2024.pptx"),
        readme_path
    ]:
        if os.path.exists(path):
            zipf.write(path, os.path.basename(path))
zip_path
